In [ ]:
# %load model.py
import inspect
import os

import numpy as np
import tensorflow as tf
import time

VGG_MEAN = [103.939, 116.779, 123.68]


class VGG16:
    def __init__(self, vgg16_npy_path, classes=1, shape=(224,224,3)):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """

        # input information
        self.H, self.W, self.C = shape
        self.classes = classes
        self.data_dict = np.load(vgg16_npy_path, encoding='latin1').item()
        print("npy file loaded")

    def build(self):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """

        # input placeholder
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.y = tf.placeholder(tf.float32, [None, self.classes])
        self.w = tf.placeholder(tf.float32, [None, self.classes])
        
        rgb_scaled = self.x

        # normalize input by VGG_MEAN
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        assert   red.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert green.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert  blue.get_shape().as_list()[1:] == [self.H, self.W, 1]

        self.x = tf.concat(axis=3, values=[
              blue - VGG_MEAN[0],
             green - VGG_MEAN[1],
               red - VGG_MEAN[2],
        ])
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]

        start_time = time.time()
        print("build model started")

        assert self.x.get_shape().as_list()[1:] == [224, 224, 3]

        self.conv1_1 = self.conv_layer(self.x, "conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1, "conv1_2")
        self.pool1 = self.max_pool(self.conv1_2, 'pool1')

        self.conv2_1 = self.conv_layer(self.pool1, "conv2_1")
        self.conv2_2 = self.conv_layer(self.conv2_1, "conv2_2")
        self.pool2 = self.max_pool(self.conv2_2, 'pool2')

        self.conv3_1 = self.conv_layer(self.pool2, "conv3_1")
        self.conv3_2 = self.conv_layer(self.conv3_1, "conv3_2")
        self.conv3_3 = self.conv_layer(self.conv3_2, "conv3_3")
        self.pool3 = self.max_pool(self.conv3_3, 'pool3')

        self.conv4_1 = self.conv_layer(self.pool3, "conv4_1")
        self.conv4_2 = self.conv_layer(self.conv4_1, "conv4_2")
        self.conv4_3 = self.conv_layer(self.conv4_2, "conv4_3")
        self.pool4 = self.max_pool(self.conv4_3, 'pool4')

        self.conv5_1 = self.conv_layer(self.pool4, "conv5_1")
        self.conv5_2 = self.conv_layer(self.conv5_1, "conv5_2")
        self.conv5_3 = self.conv_layer(self.conv5_2, "conv5_3")
        self.pool5 = self.max_pool(self.conv5_3, 'pool5')

        self.flatten_input = self.flatten(self.pool5)

        self.W1 = tf.get_variable(shape=(self.flatten_input.shape[1],1024), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="W1", dtype=tf.float32)
        self.b1 = tf.get_variable(shape=1024, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="b1", dtype=tf.float32)
        fc1 = tf.nn.bias_add(tf.matmul(self.flatten_input, self.W1), self.b1)
        self.fc1 = tf.nn.relu(fc1)

        self.W2 = tf.get_variable(shape=(1024,256), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="W2", dtype=tf.float32)
        self.b2 = tf.get_variable(shape=256, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="b2", dtype=tf.float32)
        fc2 = tf.nn.bias_add(tf.matmul(self.fc1, self.W2), self.b2)
        self.fc2 = tf.nn.relu(fc2)

        self.W3 = tf.get_variable(shape=(256,1), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="W3", dtype=tf.float32)
        self.b3 = tf.get_variable(shape=1, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name="b3", dtype=tf.float32)
        self.output = tf.nn.bias_add(tf.matmul(self.fc2, self.W3), self.b3)

        self.loss = tf.losses.mean_squared_error(labels=self.y, predictions=self.output, weights=self.w)

        print(("build model finished: %ds" % (time.time() - start_time)))

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def flatten(self,bottom):
        shape = bottom.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        x = tf.reshape(bottom, [-1, dim])
        return x

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            filt = self.get_conv_filter(name)

            conv = tf.nn.conv2d(bottom, filt, [1, 1, 1, 1], padding='SAME')

            conv_biases = self.get_bias(name)
            bias = tf.nn.bias_add(conv, conv_biases)

            relu = tf.nn.relu(bias)
            return relu

    def get_conv_filter(self, name):
        return tf.get_variable(initializer=self.data_dict[name][0], name="filter")

    def get_bias(self, name):
        return tf.get_variable(initializer=self.data_dict[name][1], name="biases")


In [ ]:
import os
import cv2
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

os.environ["CUDA_VISIBLE_DEVICES"] = "3,4"
import matplotlib.pyplot as plt
%matplotlib inline

from model import VGG16

In [ ]:
original_file = '/data/put_data/timliu/kidney/pro_img/GE/meta.csv'

In [ ]:
mydirs = ['/data/put_data/timliu/kidney/pro_img/GE/clahe'+str(i)+'_right' for i in range(5,26,5)]+['/data/put_data/timliu/kidney/pro_img/GE/clahe'+str(i)+'_left' for i in range(5,26,5)]

In [ ]:
ori = pd.read_csv(original_file, index_col = False)
df = ori.copy()
if len(mydirs)>0:
    for dirn in mydirs:
        tmp = pd.read_csv(dirn+'/df_with_labels.csv', index_col = False)
        print(tmp.shape)
        df = df.append(tmp)

df = df[(df.length != 'Suspension')].reset_index()

ori = ori[(ori.length != 'Suspension')].reset_index()

df.length = df.length.astype('float')
print(df.shape)
print(ori.shape)
df.head()

In [ ]:
ratio = 0.75
random.seed(45)
train_id = []
test_id = []
for i in range(0, 34):
    df_sub = df[df['egfr_mdrd'].between(5*i, 5*(i+1), inclusive = (True, False))]
    d_list = pd.Series.tolist(df_sub['uid_date'])
    d_list = list(set(d_list))
    random.shuffle(d_list)
    train_id += d_list[:int(ratio*len(d_list))]
    test_id += d_list[int(ratio*len(d_list)):]
print('all:', len(set(df.uid_date)), 'train:', len(train_id), 'test:', len(test_id))

In [ ]:
class Dataset(object):
    def __init__(self, uid, df, crop_size, resize_size):
        index = np.where(df['uid_date'].isin(uid))[0]
        images = []
        labels = []
        weights = []
        for i in index:
            img = self._load_image(df['path'][i], crop_size, resize_size)
            label = df['egfr_mdrd'][i]
            if label < 30:
                weight = 3
            elif label < 60:
                weight = 1
            elif label < 90:
                weight = 3
            else:
                weight = 9
            images.append(img)
            labels.append([label])
            weights.append([weight])
        self.images = np.array(images)
        self.labels = np.array(labels)
        self.weights = np.array(weights)
    
    def _load_image(self, path, crop_size, resize_size):
        # load image
        img = cv2.imread(path,1)
        # we crop image from center
        yy = int((img.shape[0] - crop_size) / 2)
        xx = int((img.shape[1] - crop_size) / 2)
        crop_img = img[yy: yy + crop_size, xx: xx + crop_size]
        # resize to 224, 224
        resized_img = cv2.resize(crop_img,(resize_size,resize_size),interpolation=cv2.INTER_AREA)
        return resized_img
    
    def shuffle(self):
        idx = np.random.permutation(self.images.shape[0])
        self.images = self.images[idx,:,:,:]
        self.labels = self.labels[idx]
        self.weights = self.weights[idx]

In [ ]:
def vgg16_train(model, train, test, init_from, save_dir, batch_size=64, epoch=300, early_stop_patience=25):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    checkpoint_path = os.path.join(save_dir, 'model.ckpt')

    with tf.Session() as sess:
        print(tf.trainable_variables())
        
        # hyper parameters
        learning_rate =  5e-4 #adam
        min_delta = 0.0001

        # recorder
        epoch_counter = 0
        loss_history = []
        val_loss_history = []

        # optimizer
        opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = opt.minimize(model.loss)
        
        # saver 
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=2)
        
        sess.run(tf.global_variables_initializer())

        # progress bar
        ptrain = IntProgress()
        pval = IntProgress()
        display(ptrain)
        display(pval)
        ptrain.max = int(train.images.shape[0]/batch_size)
        pval.max = int(test.images.shape[0]/batch_size)

        # reset due to adding a new task
        patience_counter = 0
        current_best_val_loss = 100000 # a large number
        

        # train start
        while(patience_counter < early_stop_patience):
            stime = time.time()
            bar_train = Bar('Training', max=int(train.images.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            bar_val =  Bar('Validation', max=int(test.images.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            
            # training an epoch
            train_loss = 0
            for i in range(int(train.images.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                
                _, loss = sess.run([train_op, model.loss],
                                   feed_dict={model.x: train.images[st:ed,:],
                                              model.y: train.labels[st:ed,:],
                                              model.w: train.weights[st:ed,:]
                                             })
                train_loss += loss
                ptrain.value +=1
                ptrain.description = "Training %s/%s" % (i, ptrain.max)
                bar_train.next()
            
            train_loss /= ptrain.max
            
            val_loss = 0

            for i in range(int(test.images.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                
                loss = sess.run(model.loss,
                                   feed_dict={model.x: test.images[st:ed,:],
                                              model.y: test.labels[st:ed,:],
                                              model.w: np.expand_dims(np.repeat(1.0,batch_size),axis=1)
                                             })
                val_loss += loss
                pval.value +=1
                pval.description = "Training %s/%s" % (i, pval.max)
                bar_val.next()
                
            val_loss /= pval.max
            
            if (current_best_val_loss - val_loss) > min_delta:
                current_best_val_loss = val_loss
                patience_counter = 0
                saver.save(sess, checkpoint_path, global_step=epoch_counter)
                print("reset early stopping and save model into %s at epoch %s" % (checkpoint_path,epoch_counter))
            else:
                patience_counter += 1

            # shuffle Xtrain and Ytrain in the next epoch
            train.shuffle()
            
            loss_history.append(train_loss)
            val_loss_history.append(val_loss)

            ptrain.value = 0
            pval.value = 0
            bar_train.finish()
            bar_val.finish()
            print("Epoch %s (%s), %s sec >> train-loss: %.4f, val-loss: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), train_loss, val_loss))
            
            # epoch end
            epoch_counter += 1
            if epoch_counter >= epoch:
                break
        res = pd.DataFrame({"epoch":range(0,len(loss_history)), "loss":loss_history, "val_loss":val_loss_history})
        res.to_csv(os.path.join(save_dir,"history.csv"), index=False)
        print("end training")

def plot_making(save_dir, true, pred, types):
    from scipy.stats.stats import pearsonr
    from sklearn.metrics import mean_absolute_error, r2_score
    cor = pearsonr(true, pred)[0]
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred) 
    plt.figure(0)
    plt.scatter(true, pred, alpha = .15, s = 20)
    plt.xlabel('True_Y')
    plt.ylabel('Pred_Y')
    plt.title(" data \n" + "MAE = %4f; Cor = %4f; R2 = %4f; #samples = %d" % (mae, cor, r2, len(true)))
    plt.savefig(save_dir + types + "_plot_scatter.png" , dpi = 200)
    plt.show()
    plt.close()
    
    hist = pd.read_csv(os.path.join(save_dir, "history.csv"))
    
    plt.figure(0)
    hist.plot(x='epoch', markevery=5)
    plt.xlabel('Epoch')
    plt.savefig(save_dir+"history.png",dpi=200)
    plt.show()
    plt.close()
    
def save_plot(model, save_dir, test, batch_size=64):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(save_dir)
        pred = []
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            sess.run(tf.global_variables())
            for i in range(test.images.shape[0]):
                output = sess.run(model.output,
                                   feed_dict={model.x: test.images[i:(i+1),:],
                                              model.y: test.labels[i:(i+1),:],
                                              model.w: [[1.0]]
                                             })

                pred.append(output)
            pred = np.reshape(pred,newshape=(-1,1))
            plot_making(save_dir, test.labels, pred, types="test")
            
            print("mean square error: %.4f" % np.mean(np.square(test.labels-pred)))

In [ ]:
crop_size = 350
resize_size = 224
train = Dataset(train_id, df, crop_size, resize_size)
test  = Dataset(test_id, ori, crop_size, resize_size)

In [ ]:
print("training dataset: ", train.images.shape)
print("test dataset: ", test.images.shape)

In [ ]:
save_dir = "tf_model"
init_from = "vgg16.npy"

In [ ]:
# with tf.variable_scope("test") as scope:
vgg16 = VGG16(vgg16_npy_path=init_from)
vgg16.build()

In [ ]:
vgg16_train(vgg16, train, test, save_dir=save_dir, init_from=init_from, batch_size=64, epoch=1, early_stop_patience=25)

In [ ]:
save_plot(vgg16, save_dir, test)